# Working with TESS Data

This notebook shows how to load and analyze real TESS data.

**Topics:** Loading TESS data, preprocessing, transit search

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from transitkit.io import load_tess_data_advanced
from transitkit.core import find_transits_bls_advanced
from transitkit.analysis import detrend_light_curve_gp

## 1. Load TESS Data

We'll load data for a known exoplanet host star.

In [ ]:
# Load TESS data (this requires internet connection)
# Using WASP-18 as example (TIC 100100827)
target = 'TIC 100100827'

try:
    lc_collection = load_tess_data_advanced(target, sectors=[2, 3])
    print(f'Loaded {len(lc_collection)} light curves')
except Exception as e:
    print(f'Could not load data: {e}')
    print('Using synthetic data instead...')
    lc_collection = None

In [ ]:
# If we have real data, use it; otherwise create synthetic
if lc_collection and len(lc_collection) > 0:
    lc = lc_collection[0]
    time = lc.time.value
    flux = lc.flux.value
    flux_err = lc.flux_err.value if hasattr(lc, 'flux_err') else None
    print(f'Using TESS Sector {lc.sector} data')
else:
    # Synthetic fallback
    from transitkit.core import generate_transit_signal_mandel_agol, add_noise
    time = np.linspace(0, 27, 5000)  # ~1 TESS sector
    flux = generate_transit_signal_mandel_agol(time, period=0.94, t0=0.5, depth=0.009)
    flux = add_noise(flux, 0.0005)
    flux_err = None
    print('Using synthetic data (WASP-18 b like)')

## 2. Plot Raw Light Curve

In [ ]:
plt.figure(figsize=(14, 4))
plt.plot(time, flux, 'k.', ms=0.5, alpha=0.5)
plt.xlabel('Time (BTJD)')
plt.ylabel('Normalized Flux')
plt.title(f'Light Curve ({len(time)} points)')
plt.show()

## 3. Detect Transit

In [ ]:
# Search for transits
result = find_transits_bls_advanced(
    time, flux, flux_err,
    min_period=0.3,
    max_period=10.0
)

print('Detection Results:')
print(f"  Period:   {result['period']:.6f} days")
print(f"  T0:       {result['t0']:.4f}")
print(f"  Depth:    {result['depth']*1e6:.0f} ppm")
print(f"  Duration: {result['duration']*24:.2f} hours")
print(f"  SNR:      {result['snr']:.1f}")

## 4. Phase-Folded View

In [ ]:
# Phase fold
phase = ((time - result['t0']) / result['period']) % 1
phase[phase > 0.5] -= 1

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Full phase
axes[0].plot(phase, flux, 'k.', ms=1, alpha=0.3)
axes[0].set_xlabel('Phase')
axes[0].set_ylabel('Flux')
axes[0].set_title('Full Phase')

# Zoom on transit
axes[1].plot(phase, flux, 'k.', ms=1, alpha=0.3)
axes[1].set_xlabel('Phase')
axes[1].set_ylabel('Flux')
axes[1].set_xlim(-0.05, 0.05)
axes[1].set_title('Transit Zoom')

plt.tight_layout()
plt.show()

## 5. Optional: GP Detrending

Remove stellar variability before analysis.

In [ ]:
# Create in-transit mask
phase_transit = ((time - result['t0']) / result['period']) % 1
phase_transit[phase_transit > 0.5] -= 1
in_transit = np.abs(phase_transit) < (result['duration'] / result['period'] / 2 * 1.5)

try:
    detrended, trend, _ = detrend_light_curve_gp(time, flux, in_transit_mask=in_transit)
    
    fig, axes = plt.subplots(2, 1, figsize=(14, 6), sharex=True)
    axes[0].plot(time, flux, 'k.', ms=0.5, alpha=0.5, label='Original')
    axes[0].plot(time, trend, 'r-', lw=1, label='GP Trend')
    axes[0].legend()
    axes[0].set_ylabel('Flux')
    
    axes[1].plot(time, detrended, 'k.', ms=0.5, alpha=0.5)
    axes[1].set_xlabel('Time')
    axes[1].set_ylabel('Detrended Flux')
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f'GP detrending skipped: {e}')

## Summary

This notebook showed:
1. Loading TESS data with `load_tess_data_advanced`
2. Running BLS detection
3. Creating phase-folded plots
4. Optional GP detrending

**Tip:** Try different TIC IDs for known planets!